In [1]:
import sys
sys.path.insert(0, '/home/louchao/project/semantic_parsing_qcfg/')
from src.models.tgt_parser.struct.pcfg_rdp import PCFGRandomizedDP
from src.models.tgt_parser.struct.pcfg import PCFG
import torch

In [5]:

torch.random.manual_seed(1)

B, N, T, NT = 1, 60, 80, 80
device = "cuda"
params = {
    "term": torch.randn(B, N, T, device=device)
    .log_softmax(-1)
    .requires_grad_(True),
    "root": torch.randn(B, NT, device=device).log_softmax(-1).requires_grad_(True),
    "rule": torch.randn(B, NT, (NT + T) ** 2, device=device)
    .log_softmax(-1)
    .view(B, NT, NT + T, NT + T)
    .requires_grad_(True),
}
lens = torch.tensor([N], dtype=torch.long, device=device)

In [4]:
# sanity check 
pcfg = PCFGRandomizedDP(3, 3, 0)
out = torch.zeros(B, device=device)
print(pcfg(params, lens))
for _ in range(100):
    out += pcfg(params, lens)
print(out / 100)

pcfg = PCFG()
print(pcfg(params, lens))
print(pcfg(params, lens, decode=True))


tensor([131.8035, 129.5500, 129.1859, 125.3113], device='cuda:0',
       grad_fn=<NegBackward0>)
tensor([131.8895, 129.5937, 129.3460, 125.0959], device='cuda:0',
       grad_fn=<DivBackward0>)
tensor([131.8672, 129.5899, 129.3283, 125.1133], device='cuda:0',
       grad_fn=<NegBackward0>)
[[(0, 0, 1), (1, 1, 3), (2, 2, 1), (3, 3, 2), (4, 4, 7), (5, 5, 4), (6, 6, 2), (7, 7, 4), (8, 8, 5), (9, 9, 2), (10, 10, 7), (11, 11, 1), (12, 12, 4), (13, 13, 4), (14, 14, 5), (15, 15, 0), (16, 16, 2), (17, 17, 2), (18, 18, 3), (19, 19, 6), (20, 20, 6), (21, 21, 5), (22, 22, 4), (23, 23, 3), (24, 24, 2), (25, 25, 7), (26, 26, 6), (27, 27, 0), (28, 28, 7), (29, 29, 0), (30, 30, 2), (31, 31, 6), (32, 32, 3), (33, 33, 5), (34, 34, 0), (35, 35, 4), (36, 36, 3), (37, 37, 4), (38, 38, 4), (39, 39, 2), (40, 40, 3), (41, 41, 0), (42, 42, 3), (43, 43, 1), (44, 44, 7), (45, 45, 6), (46, 46, 1), (47, 47, 1), (48, 48, 6), (49, 49, 2), (50, 50, 6), (51, 51, 1), (52, 52, 0), (53, 53, 2), (54, 54, 7), (55, 55, 0),

In [3]:
from pytorch_memlab import LineProfiler

In [6]:
pcfg = PCFGRandomizedDP(10, 10, 0)
with LineProfiler(pcfg.__call__) as prof:
    pcfg(params, lens, decode=True)
prof.display()

/home/louchao/anaconda3/envs/neural-qcfg/lib/python3.9/site-packages/pytorch_memlab/line_profiler/line_records.py:60: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  records = (_accumulate_line_records(raw_line_records)
/home/louchao/anaconda3/envs/neural-qcfg/lib/python3.9/site-packages/pytorch_memlab/line_profiler/line_records.py:189: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged = merged.drop('code', 1, level=0)
/home/louchao/anaconda3/envs/neural-qcfg/lib/python3.9/site-packages/pytorch_memlab/line_profiler/line_records.py:215: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  html[qual_name] = (style
/home/louchao/anaconda3/envs/neural-qcfg/lib/python3.9/site-packages/pytorch_memlab/line_profiler/line_records.py:215: FutureWarning: this method is deprecated in favour of

active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
5.55G,5.66G,20,"def __call__(self, params, lens, decode=False, marginal=False):"
,,21,# terms: bsz x seqlen x pt
,,22,# rules: bsz x nt x (nt+pt) x (nt+pt)
,,23,# roots: bsz x nt
,,24,
5.55G,5.66G,25,"topk, sample_size, smooth = self.topk, self.sample_size, self.smooth"
5.55G,5.66G,26,"batch, N, T = params[""term""].shape"
5.55G,5.66G,27,N += 1


In [7]:
%debug

> /home/louchao/project/semantic_parsing_qcfg/src/models/tgt_parser/struct/pcfg_rdp.py(128)sample()
    126     b, n, c = score.shape
    127     proposal_p = score.detach().softmax(-1)
--> 128     _, topk_ind = torch.topk(proposal_p, topk, dim=-1, sorted=False)
    129     topk_score = score.gather(-1, topk_ind)
    130 

10
10
torch.Size([4, 59, 8])
